In [34]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [35]:
# Load dataset
df = pd.read_csv(r"C:\Users\asus\Garbage-Sense\trash_fill_data .csv")  # Update path if needed

# Display first few rows
df.head()


,Time (hrs),Fill Percentage (%),Fill Level (cm),Day,Hour,Fill Rate (cm/hr),Previous Fill Percentage (%),Time Since Emptying (hrs),Time of Day_Morning,Time of Day_Afternoon,Time of Day_Evening
0,0.000000,4.518090,1.355427,0,0,0.000000,0.000000,0.000000,0,0,0
1,0.163004,0.124241,0.037272,0,0,-8.086628,4.518090,0.163004,0,0,0
2,0.326008,7.452207,2.235662,0,0,13.486703,0.124241,0.163004,0,0,0
3,0.489013,9.844623,2.953387,0,0,4.403105,7.452207,0.163004,0,0,0
4,0.652017,5.157353,1.547206,0,0,-8.626653,9.844623,0.163004,0,0,0


In [ ]:
# "Cycle Time Remaining (hrs) grouping for calculateions 
df["Cycle Time Remaining (hrs)"] = df.groupby("Day")["Time (hrs)"].transform("max") - df["Time (hrs)"]

# Ensure values stay within a valid range (0 to 9 hours)
df["Cycle Time Remaining (hrs)"] = df["Cycle Time Remaining (hrs)"].clip(0, 9)



In [42]:
# Ensure "Adjusted Fill Rate (cm/hr)" exists
if "Fill Rate (cm/hr)" in df.columns:
    df["Adjusted Fill Rate (cm/hr)"] = df["Fill Rate (cm/hr)"]

    # Add variation for lower fill percentages to simulate real-world behavior
    low_fill_mask = df["Fill Percentage (%)"] < 20
    df.loc[low_fill_mask, "Adjusted Fill Rate (cm/hr)"] *= np.random.uniform(0.7, 1.3, low_fill_mask.sum())
else:
    print("⚠️ Warning: 'Fill Rate (cm/hr)' is missing from the dataset!")

# Ensure feature list is correct
feature_columns = [
    "Fill Percentage (%)", "Fill Level (cm)", "Fill Rate (cm/hr)", 
    "Previous Fill Percentage (%)", "Time Since Emptying (hrs)", "Adjusted Fill Rate (cm/hr)"
]
feature_columns.extend([col for col in df.columns if "Time of Day" in col])

# Check for missing features
missing_features = [col for col in feature_columns if col not in df.columns]
if missing_features:
    print(f"⚠️ Warning: The following features are missing from the dataset: {missing_features}")


In [43]:
# Define features and target
X = df[feature_columns]  # Features
y = df["Cycle Time Remaining (hrs)"]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, min_samples_split=5)
rf_model.fit(X_train, y_train)

print("✅ Model Retrained Successfully!")


✅ Model Retrained Successfully!


In [44]:
# Function to Predict Time Remaining
def predict_fill_time(fill_percentage):
    fill_level = (fill_percentage / 100) * 30
    estimated_fill_rate = df["Fill Rate (cm/hr)"].median()
    previous_fill_percentage = fill_percentage - estimated_fill_rate
    estimated_time_since_emptying = df.loc[
        (df["Fill Percentage (%)"] >= fill_percentage - 1) & 
        (df["Fill Percentage (%)"] <= fill_percentage + 1),
        "Time Since Emptying (hrs)"
    ].median()

    # Average values for categorical features
    time_of_day_features = df.filter(like="Time of Day").mean().to_dict()

    # Create input data
    input_data = pd.DataFrame([{
        "Fill Percentage (%)": fill_percentage,
        "Fill Level (cm)": fill_level,
        "Fill Rate (cm/hr)": estimated_fill_rate,
        "Previous Fill Percentage (%)": previous_fill_percentage,
        "Time Since Emptying (hrs)": estimated_time_since_emptying,
        "Adjusted Fill Rate (cm/hr)": estimated_fill_rate,
        **time_of_day_features
    }])

    # Predict and round to 2 decimal places
    return round(rf_model.predict(input_data)[0], 2)


In [50]:
print("Prediction for 61.1% fill:", predict_fill_time(61.1))
print("Prediction for 15% fill:", predict_fill_time(15.0))
print("Prediction for 4% fill:", predict_fill_time(4.0))
print("Prediction for 98% fill:", predict_fill_time(98.2))



Prediction for 61.1% fill: 3.07
Prediction for 15% fill: 7.06
Prediction for 4% fill: 7.32
Prediction for 98% fill: 0.21
